In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    if "cassavav16" in dirname:
        for f in filenames:
            print(f)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/cassavav12
/kaggle/input/cassavav07
/kaggle/input/cassavav05
/kaggle/input/cnnstackingv01
/kaggle/input/cassavav01
/kaggle/input/cassavav01/utils
/kaggle/input/cassavav01/weights
/kaggle/input/cassavav01/pytorch-image-models
/kaggle/input/cassavav01/pytorch-image-models/tests
/kaggle/input/cassavav01/pytorch-image-models/results
/kaggle/input/cassavav01/pytorch-image-models/.github
/kaggle/input/cassavav01/pytorch-image-models/.github/ISSUE_TEMPLATE
/kaggle/input/cassavav01/pytorch-image-models/.github/workflows
/kaggle/input/cassavav01/pytorch-image-models/docs
/kaggle/input/cassavav01/pytorch-image-models/docs/javascripts
/kaggle/input/cassavav01/pytorch-image-models/notebooks
/kaggle/input/cassavav01/pytorch-image-models/timm
/kaggle/input/cassavav01/pytorch-image-models/timm/utils
/kaggle/input/cassavav01/pytorch-image-models/timm/scheduler
/kaggle/input/cassavav01/pytorch-image-models/timm/optim
/kaggle/input/cassavav01/pytorch-image-models/timm/loss
/k

In [2]:
!pip install /kaggle/input/cassavav01/pytorch-image-models > /dev/null

In [3]:
!cp -r /kaggle/input/cassavav01/utils . 

In [4]:
from collections import defaultdict
import copy
import random
import numpy as np
import os
import shutil
from urllib.request import urlretrieve
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
cudnn.benchmark = True
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import timm
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from utils import Mixup, RandAugment
from PIL import Image
import torch.nn.functional as F
SEED = 42

In [5]:
class CNNStackModel(nn.Module):
    def __init__(self, num_classes, num_channels):
        super(CNNStackModel, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, 256, kernel_size=(1,3), stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=(3,1), stride=1, padding=0),
            nn.ReLU(inplace=True),
        )
        self.fc1 = nn.Linear(512, 1024, bias=True)
        self.fc2 = nn.Linear(1024, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

In [6]:
def seed_everything(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(SEED)

In [7]:
root = os.path.join(os.environ["HOME"], "/kaggle/input/cassava-leaf-disease-classification")
os.listdir(root)

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']

In [8]:
train = pd.read_csv(f'{root}/train.csv')
test = pd.read_csv(f'{root}/sample_submission.csv')
label_map = pd.read_json(f'{root}/label_num_to_disease_map.json', 
                         orient='index')
display(train.head())
display(test.head())
display(label_map)

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


,image_id,label
0,2216849948.jpg,4


,0
0,Cassava Bacterial Blight (CBB)
1,Cassava Brown Streak Disease (CBSD)
2,Cassava Green Mottle (CGM)
3,Cassava Mosaic Disease (CMD)
4,Healthy


In [9]:
models_name = ["resnest26d", "resnest50d" , "tf_efficientnet_b4_ns"]


WEIGHTS_26 = [

    "../input/cassavav05/resnest26d_fold0_best_epoch_19_final_3rd.pth",
    "../input/cassavav05/resnest26d_fold1_best_epoch_7_final_2nd.pth",
    "../input/cassavav05/resnest26d_fold2_best_epoch_4_final_2nd.pth",
    "../input/cassavav05/resnest26d_fold3_best_epoch_10_final_3rd.pth",
    "../input/cassavav05/resnest26d_fold4_best_epoch_6_final_3rd.pth"]
    #
WEIGHTS_50 = [
    "../input/cassavav07/resnest50d_fold0_best_epoch_10_final_3rd.pth",
    "../input/cassavav09/resnest50d_fold1_best_epoch_8_final_5th_pseudo.pth",
    "../input/cassavav06/resnest50d_fold2_best_epoch_22_final_2nd.pth",
    "../input/cassavav07/resnest50d_fold3_best_epoch_1_final_3rd.pth",
    "../input/cassavav09/resnest50d_fold4_best_epoch_1_final_5th_pseudo.pth"]
# 
WEIGHTS_b4 = [

    "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold0_best_epoch_25_final_3rd.pth",
    "../input/effb45thfold1/tf_efficientnet_b4_ns_fold1_best_epoch_26_final_5th.pth",
    "../input/effb43rdfold2/tf_efficientnet_b4_ns_fold2_best_epoch_29_final_3rd.pth",
    "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold3_best_epoch_14_final_2nd.pth",
    "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold4_best_epoch_20_final_3rd.pth",
]

SKWeights = [
        "../input/cnnstackingv01/cnn-stack_fold0_best_epoch_30.pth",
        "../input/cnnstackingv01/cnn-stack_fold1_best_epoch_1.pth",
        "../input/cnnstackingv01/cnn-stack_fold2_best_epoch_16.pth",
        "../input/cnnstackingv01/cnn-stack_fold3_best_epoch_23.pth",
        "../input/cnnstackingv01/cnn-stack_fold4_best_epoch_27.pth",
]


params = {
    "visualize": False,
    "debug":False,
    "fold": [0,1,2,3,4],
    "load_pretrained": True,
    "image_size": 512,
    "num_classes": 5,
    "device": "cuda",
    "batch_size": 32,
    "num_workers": 2,
    "drop_block": 0.2,
    "drop_rate": 0.2,
    "tta": True,
    "kfold_pred":True,
    "error_fix":False,
    "ensemble":False
}

In [10]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None, valid_test=False):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        self.valid_test = valid_test
        if self.valid_test:
            self.labels = df['label'].values  
        else:
            assert ValueError("Test data does not have annotation, plz check!")
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{root}/test_images/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if isinstance(self.transform, list):
            outputs = {'images':[],
                       'labels':[]}
             #image0 = transforms.ToPILImage()(image)
             #image0 = self.transform[0](image0)

            for trans in self.transform:
                augmented = trans(image=image)
                image_aug = augmented['image']
                outputs["images"].append(image_aug)
                del image_aug
                
            if self.valid_test:
                label = torch.tensor(self.labels[idx]).long()
                outputs['labels'] = len(self.transform)*[label]
            else:
                outputs['labels'] = len(self.transform)*[-1]
                
            return outputs
        else:
            augmented = self.transform(image=image)
            image = augmented['image'] 
        return image

In [11]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [12]:
def declare_pred_model(name, weight):
    if "efficientnet" in name:
        model = timm.create_model(
                name,
                pretrained=False,
                num_classes=5, 
                drop_rate=0.2, 
                drop_path_rate=0.3)
    else:
        model = timm.create_model(
                name,
                pretrained=False,
                num_classes=5,
                drop_rate=0.2)

    model = model.to(params["device"])
    model = torch.nn.DataParallel(model) 
    state_dict = torch.load(weight)
    print(f"Load pretrained model: {name} ",state_dict["preds"])
    model.load_state_dict(state_dict["model"])
    best_acc = state_dict["preds"]   
    return model.eval()    

In [13]:
transform_tta0 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)

transform_tta1 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.HorizontalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)
transform_tta2 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),        
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.VerticalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
     ToTensorV2(),
    ]
)
transform_tta3 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.RandomRotate90(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),     
     ToTensorV2(),
    ]
)
##### Test TTA with Five Crops
test_transform_tta = [transform_tta0, transform_tta1, transform_tta2, transform_tta3]

#debug
if params['debug']:
    print("In debug mode")
    test = pd.concat(100*[test])

if params["tta"]:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta)
else:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta[0])
    
test_pred_loader = DataLoader(
    test_pred_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params['num_workers'], pin_memory=True,
)

In [14]:
test.head

<bound method NDFrame.head of          image_id  label
0  2216849948.jpg      4>

In [15]:
def visualize_tta(data, pred):     
    unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    if params["tta"]:
        figure, ax = plt.subplots(nrows=1, ncols=num_tta, figsize=(12, 6))
        for i, image in enumerate(data["images"]):
            image = (unorm(image[0]).cpu().numpy()*255).astype(int)
            ax.ravel()[i].imshow(image.transpose(2,1,0))
            ax.ravel()[i].set_title(str(pred), color='GREEN')
            ax.ravel()[i].set_axis_off()
        plt.tight_layout()
        plt.show() 
    else:
        image = (unorm(data).cpu().numpy()*255).astype(int)
        imgplot = plt.imshow(image[0].transpose(2,1,0))
        imgplot = plt.title(str(pred), color='GREEN')
        plt.show()

In [16]:
def gmean(input_x, dim):
    log_x = torch.log(input_x)
    return torch.exp(torch.mean(log_x, dim=dim))

In [17]:
def tta_stack_validate(loader, model, params, fold_idx):
    model.eval()
    stream = tqdm(loader)
    preds = []
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            tta_output = [] 
            for i, image in enumerate(data["images"]):
                logit = model(image)
                tta_output.append(logit)
            preds.append(torch.stack(tta_output, dim=0).permute(1,0,2))
    return torch.cat(preds, dim=0)


In [18]:
r26_logit_preds = {
        "logits":[],
    }
r50_logit_preds = {
        "logits":[],
    }
eb4_logit_preds = {
        "logits":[],
    }
for fold_idx in range(5):
    r26_model = declare_pred_model(models_name[0], WEIGHTS_26[fold_idx])
    r26_outputs = tta_stack_validate(test_pred_loader, r26_model, params, fold_idx)
    r26_logit_preds["logits"].append(r26_outputs)
    del r26_outputs
del r26_model

for fold_idx in range(5):
    eb4_model = declare_pred_model(models_name[2], WEIGHTS_b4[fold_idx])    
    eb4_outputs = tta_stack_validate(test_pred_loader, eb4_model, params, fold_idx)
    eb4_logit_preds["logits"].append(eb4_outputs)
    del eb4_outputs
del eb4_model
    
for fold_idx in range(5):
    r50_model = declare_pred_model(models_name[1], WEIGHTS_50[fold_idx])
    r50_outputs = tta_stack_validate(test_pred_loader, r50_model, params, fold_idx)
    r50_logit_preds["logits"].append(r50_outputs)
    del r50_outputs
del r50_model

del test_pred_dataset, test_pred_loader

  0%|          | 0/1 [00:00<?, ?it/s]

Load pretrained model: resnest26d  0.8864


  0%|          | 0/1 [00:00<?, ?it/s]

Load pretrained model: resnest26d  0.8997


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Load pretrained model: resnest50d  0.8906


  0%|          | 0/1 [00:00<?, ?it/s]

Load pretrained model: resnest50d  0.892


100%|██████████| 1/1 [00:00<00:00,  4.87it/s]


In [19]:
r26_logit_preds["logits"][0].shape

torch.Size([1, 4, 5])

In [20]:
class StackTestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = self.df[idx]
        if self.transform:
            augmented = self.transform(image=image.cpu().numpy().transpose(1,2,0))
            image = augmented['image']
        return image

In [21]:
stack_transform = A.Compose(
[
    ToTensorV2(),
])
stack_test_data = torch.stack([torch.stack(r26_logit_preds["logits"], dim=0).mean(dim=0),
                         torch.stack(r50_logit_preds["logits"], dim=0).mean(dim=0),
                         torch.stack(eb4_logit_preds["logits"], dim=0).mean(dim=0)
                        ])
stack_test_data = stack_test_data.permute(1,0,2,3)
stack_dataset = StackTestDataset(stack_test_data, transform=stack_transform)
stack_data_loader = DataLoader(
        stack_dataset, batch_size=128, shuffle=False, pin_memory=True,
    )
stack_test_data.shape

torch.Size([1, 3, 4, 5])

In [22]:
print(f"***************** Stack models prediction *****************")
stack_models = []
for ckpt in range(5):
    sm = CNNStackModel(params["num_classes"], len(models_name))
    sm = sm.to(params["device"])
    sm = torch.nn.DataParallel(sm) 
    state_dict = torch.load(SKWeights[ckpt])
    sm.load_state_dict(state_dict["model"])
    acc = state_dict["preds"]
    print(f"Load stacking model: {SKWeights[ckpt]}, acc: {acc}")
    
    stack_models.append(sm.eval())
    del sm


***************** Stack models prediction *****************
Load stacking model: ../input/cnnstackingv01/cnn-stack_fold0_best_epoch_30.pth, acc: 0.8914
Load stacking model: ../input/cnnstackingv01/cnn-stack_fold1_best_epoch_1.pth, acc: 0.9002
Load stacking model: ../input/cnnstackingv01/cnn-stack_fold2_best_epoch_16.pth, acc: 0.8998
Load stacking model: ../input/cnnstackingv01/cnn-stack_fold3_best_epoch_23.pth, acc: 0.8902
Load stacking model: ../input/cnnstackingv01/cnn-stack_fold4_best_epoch_27.pth, acc: 0.8987


In [23]:
final_out = []
for img in tqdm(stack_data_loader):
    kfold_out = [torch.softmax(model(img.cuda()), dim = 1) for model in stack_models]
    final_out.extend(torch.stack(kfold_out, dim=0).mean(dim=0))

final_out = torch.argmax(torch.stack(final_out, dim=0), dim =1).cpu().numpy()

100%|██████████| 1/1 [00:00<00:00, 49.92it/s]


In [24]:
test['label'] = final_out 
test.to_csv('submission.csv', index=False)
test.head(n=10)

,image_id,label
0,2216849948.jpg,2
